<a href="https://colab.research.google.com/github/SunkaraboinaPraveenKumar/Machine_Learning_Projects/blob/main/Anime_Movie_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain langchain_community langchain_groq chromadb gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 45.7 MB/s eta 0:00:

In [2]:
import pandas as pd
from langchain.chains import RetrievalQA
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders.csv_loader import CSVLoader

In [4]:
anime=pd.read_csv("/content/anime_with_synopsis.csv")

In [5]:
anime.dropna()

,MAL_ID,Name,Score,Genres,sypnopsis
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space","In the year 2071, humanity has colonized sever..."
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space","other day, another bounty—such is the life of ..."
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen","Vash the Stampede is the man with a $$60,000,0..."
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",ches are individuals with special powers like ...
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",It is the dark century and the people are suff...
...,...,...,...,...,...
264,296,Dragon Drive,6.73,"Action, Sci-Fi, Adventure, Comedy, Fantasy, Sc...","If there's one word to describe Reiji Ozora, i..."
265,297,Grenadier: Hohoemi no Senshi,6.72,"Action, Ecchi, Adventure, Comedy, Shounen",ushuna is a blonde and very beautiful Senshi (...
266,298,.hack//Tasogare no Udewa Densetsu,6.60,"Adventure, Comedy, Fantasy, Game, Sci-Fi, Shounen","g the legendary characters ""Kite"" and ""Black R..."
267,299,.hack//Liminality,6.61,"Game, Mystery, Sci-Fi","hile playing the newly released MMORPG ""The Wo..."


In [7]:
from langchain_groq import ChatGroq

In [8]:
anime['combined_df']=anime.apply(lambda row:f"Title: {row['Name']}. Overview: {row['sypnopsis']}. Genres: {row['Genres']}", axis=1)

In [9]:
anime

,MAL_ID,Name,Score,Genres,sypnopsis,combined_df
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space","In the year 2071, humanity has colonized sever...",Title: Cowboy Bebop. Overview: In the year 207...
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space","other day, another bounty—such is the life of ...",Title: Cowboy Bebop: Tengoku no Tobira. Overvi...
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen","Vash the Stampede is the man with a $$60,000,0...",Title: Trigun. Overview: Vash the Stampede is ...
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",ches are individuals with special powers like ...,Title: Witch Hunter Robin. Overview: ches are ...
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",It is the dark century and the people are suff...,Title: Bouken Ou Beet. Overview: It is the dar...
...,...,...,...,...,...,...
264,296,Dragon Drive,6.73,"Action, Sci-Fi, Adventure, Comedy, Fantasy, Sc...","If there's one word to describe Reiji Ozora, i...",Title: Dragon Drive. Overview: If there's one ...
265,297,Grenadier: Hohoemi no Senshi,6.72,"Action, Ecchi, Adventure, Comedy, Shounen",ushuna is a blonde and very beautiful Senshi (...,Title: Grenadier: Hohoemi no Senshi. Overview:...
266,298,.hack//Tasogare no Udewa Densetsu,6.60,"Adventure, Comedy, Fantasy, Game, Sci-Fi, Shounen","g the legendary characters ""Kite"" and ""Black R...",Title: .hack//Tasogare no Udewa Densetsu. Over...
267,299,.hack//Liminality,6.61,"Game, Mystery, Sci-Fi","hile playing the newly released MMORPG ""The Wo...",Title: .hack//Liminality. Overview: hile playi...


In [10]:
anime['combined_df'][0]

'Title: Cowboy Bebop. Overview: In the year 2071, humanity has colonized several of the planets and moons of the solar system leaving the now uninhabitable surface of planet Earth behind. The Inter Solar System Police attempts to keep peace in the galaxy, aided in part by outlaw bounty hunters, referred to as "Cowboys." The ragtag team aboard the spaceship Bebop are two such individuals. Mellow and carefree Spike Spiegel is balanced by his boisterous, pragmatic partner Jet Black as the pair makes a living chasing bounties and collecting rewards. Thrown off course by the addition of new members that they meet in their travels—Ein, a genetically engineered, highly intelligent Welsh Corgi; femme fatale Faye Valentine, an enigmatic trickster with memory loss; and the strange computer whiz kid Edward Wong—the crew embarks on thrilling adventures that unravel each member\'s dark and mysterious past little by little. Well-balanced with high density action and light-hearted comedy, Cowboy Bebo

In [11]:
anime[['combined_df']].to_csv('anime_updated.csv',index=False)

In [12]:
pd.read_csv("/content/anime_updated.csv")

,combined_df
0,Title: Cowboy Bebop. Overview: In the year 207...
1,Title: Cowboy Bebop: Tengoku no Tobira. Overvi...
2,Title: Trigun. Overview: Vash the Stampede is ...
3,Title: Witch Hunter Robin. Overview: ches are ...
4,Title: Bouken Ou Beet. Overview: It is the dar...
...,...
264,Title: Dragon Drive. Overview: If there's one ...
265,Title: Grenadier: Hohoemi no Senshi. Overview:...
266,Title: .hack//Tasogare no Udewa Densetsu. Over...
267,Title: .hack//Liminality. Overview: hile playi...


Data Loader and VectorStore

In [13]:
from google.colab import userdata
import os
os.environ['GROQ_API_KEY']=userdata.get('groq_api_key')

In [14]:
loader = CSVLoader(file_path="/content/anime_updated.csv")
data = loader.load()

text_splitter=CharacterTextSplitter(chunk_size=1000,chunk_overlap=0)
texts=text_splitter.split_documents(data)

embeddings=HuggingFaceEmbeddings()

llm=ChatGroq(
    model="llama-3.3-70b-versatile"
)

<ipython-input-14-0ddb810a8d89>:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings()
<ipython-input-14-0ddb810a8d89>:7: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings=HuggingFaceEmbeddings()


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [15]:
docs_search = Chroma.from_documents(texts, embeddings)

In [16]:
# Querying Vector DB Store for MOVIE Recommendation

In [19]:
query = "I am looking for Romantic and Love Story Kind of Movies"
docs=docs_search.similarity_search(query, k=2)

In [20]:
docs

[Document(metadata={'row': 216, 'source': '/content/anime_updated.csv'}, page_content='combined_df: Title: Ichigo 100%. Overview: One day, Manaka Junpei walks to the roof of his school and encounters a beautiful girl falling down from above him and accidentally exposing her strawberry panties. The embarrassed girl runs away before Junpei can find out whom she is. He wishes to become a filmmaker, and this whole experience seemed like it would make the perfect scene in a movie. And so he goes on a search for the girl with the strawberry panties in order to reenact it all on film. But he will soon discover that finding that one girl will not be that easy... (Source: ANN). Genres: Comedy, Ecchi, Harem, Romance, Shounen'),
 Document(metadata={'row': 177, 'source': '/content/anime_updated.csv'}, page_content='combined_df: Title: Video Girl Ai. Overview: fter a crushing rejection, the heartbroken Youta Moteuchi trudges home. On his way, a dimly lit video store catches his eye, and he ends up 

In [23]:
# Using QA retrieval
qa=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docs_search.as_retriever(),
    return_source_documents=True
)

In [24]:
query = "I am looking for Romantic and Love Story Kind of Movies"
result = qa({"query":query})

<ipython-input-24-a01b457e6416>:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa({"query":query})


In [25]:
result['result']

'Based on the provided context, here are some anime recommendations that fit the Romantic and Love Story genre:\n\n1. **Ichigo 100%** - A romantic comedy with a harem twist, where the main character searches for a girl he fell in love with at first sight.\n2. **Video Girl Ai** - A romantic story about a boy who finds love and companionship with a "video girl" who comes to life, helping him to find true love.\n3. **Pia Carrot e Youkoso!! 2 DX** - A romantic comedy that explores the development of feelings between two characters who initially dislike each other.\n4. **Boys Be...** - A slice-of-life anime that focuses on the romantic misadventures of three high school guys as they navigate love and relationships.\n\nAll of these anime have romance as a central theme, and they offer a mix of comedy, drama, and heartwarming moments. I hope you find something that interests you!'

In [26]:
result['source_documents'][0]

Document(metadata={'row': 216, 'source': '/content/anime_updated.csv'}, page_content='combined_df: Title: Ichigo 100%. Overview: One day, Manaka Junpei walks to the roof of his school and encounters a beautiful girl falling down from above him and accidentally exposing her strawberry panties. The embarrassed girl runs away before Junpei can find out whom she is. He wishes to become a filmmaker, and this whole experience seemed like it would make the perfect scene in a movie. And so he goes on a search for the girl with the strawberry panties in order to reenact it all on film. But he will soon discover that finding that one girl will not be that easy... (Source: ANN). Genres: Comedy, Ecchi, Harem, Romance, Shounen')

In [27]:
# Prompt Engineering
from langchain.prompts import PromptTemplate
template = """You are a movie recommender system that help users to find anime that match their preferences.
Use the following pieces of context to answer the question at the end.
For each question, suggest three anime, with a short description of the plot and the reason why the user migth like it.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Your response:"""

PROMPT = PromptTemplate(
    template=template,
    input_variables=["context", "question"]
)

chain_type_kwargs={"prompt":PROMPT}

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docs_search.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

query = "Give me romantic movies ever."
result = qa({"query":query})
print(result["result"])

Based on the information provided, here are three romantic anime recommendations:

1. **Kimi ga Nozomu Eien**: This anime is a heart-wrenching romance that explores the complexities of love, friendship, and tragedy. It's a beautiful and emotional story that follows the lives of high school students as they navigate love, loss, and relationships. You might like it because of its deep and realistic portrayal of romance.

2. **Video Girl Ai**: This anime is a unique blend of romance, comedy, and drama that follows the story of a heartbroken boy who finds love and companionship with a "video girl" named Ai. It's a sweet and entertaining story that explores the ups and downs of romance and relationships. You might like it because of its lighthearted and humorous take on love.

3. **Boys Be....**: This anime is a relatable and realistic portrayal of high school romance from the guy's point of view. It follows the lives of three friends as they navigate love, relationships, and growing up. Yo

In [28]:
from langchain.prompts import PromptTemplate

template_prefix = """You are a movie recommender system that help users to find anime that match their preferences.
Use the following pieces of context to answer the question at the end.
For each question, take into account the context and the personal information provided by the user.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}"""

user_info = """This is what we know about the user, and you can use this information to better tune your research:
Age: {age}
Gender: {gender}"""

template_suffix= """Question: {question}
Your response:"""

user_info = user_info.format(age = 18, gender = 'male')

COMBINED_PROMPT = template_prefix +'\n'+ user_info +'\n'+ template_suffix
print(COMBINED_PROMPT)

You are a movie recommender system that help users to find anime that match their preferences.
Use the following pieces of context to answer the question at the end.
For each question, take into account the context and the personal information provided by the user.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}
This is what we know about the user, and you can use this information to better tune your research:
Age: 18
Gender: male
Question: {question}
Your response:


In [30]:
PROMPT  =  PromptTemplate(
    template=COMBINED_PROMPT, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}

qa = RetrievalQA.from_chain_type(llm,
            chain_type = "stuff",
            retriever = docs_search.as_retriever(),
            return_source_documents = True,
                                 chain_type_kwargs = chain_type_kwargs)

query = "Which anime would you suggest for someone new to anime?"
result = qa({"query": query})
print(result['result'])

For someone new to anime, I'd suggest "Gakuen Alice". It's a heartwarming comedy that's easy to follow, with a mix of school life, supernatural powers, and adventure. The story is engaging, and the characters are relatable and likable. The genre is also quite accessible, being a mix of comedy, school, and superpower elements, which are common in many anime shows. Additionally, it's a great introduction to the Shoujo genre, which is known for its light-hearted and entertaining storylines. Overall, I think "Gakuen Alice" is a great starting point for someone new to anime, as it's an enjoyable and easy-to-watch series that can help you get a feel for the medium.


In [31]:
import gradio as gr
import pandas as pd
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.prompts import PromptTemplate

# Load anime dataset
anime = pd.read_csv("anime_with_synopsis.csv")
anime = anime.dropna()
anime['combined_info'] = anime.apply(lambda row: f"Title: {row['Name']}. Overview: {row['sypnopsis']} Genres: {row['Genres']}", axis=1)

# Save updated dataset
anime[['combined_info']].to_csv('anime_updated.csv', index=False)
loader = CSVLoader(file_path="anime_updated.csv")
data = loader.load()

# Text splitting
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

# Create embeddings
embeddings = HuggingFaceEmbeddings()
docssearch = Chroma.from_documents(texts, embeddings)
retriever = docssearch.as_retriever()

# Define LLM
llm = ChatGroq(api_key="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", model="llama-3.3-70b-specdec", temperature=0)

# Custom Prompt Template
template = """You are a movie recommender system that helps users find anime that match their preferences.
Use the following pieces of context to answer the question at the end.
For each question, suggest three anime, with a short description of the plot and the reason why the user might like it.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Your response:"""

PROMPT = PromptTemplate(template=template, input_variables=["context", "question"])
chain_type_kwargs = {"prompt": PROMPT}

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

def recommend_anime(query):
    result = qa_chain({'query': query})
    return result['result']

# Gradio interface
theme = "Respair/Shiki@1.2.1"

iface = gr.Interface(
    fn=recommend_anime,
    inputs=gr.Textbox(label="Enter your anime preference"),
    outputs=gr.Textbox(label="Recommended Anime"),
    title="Anime Movie Recommender",
    theme=theme,

)

if __name__ == "__main__":
    iface.launch()



<ipython-input-31-2926f974f3ac>:26: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()


theme_schema%401.2.1.json:   0%|          | 0.00/14.5k [00:00<?, ?B/s]

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ced47ede6ce5620564.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
